In [ ]:
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import pandas as pd

import albumentations as A

import torch
from tqdm import tqdm

import torch
import torch.nn as nn
import timm
import torchvision
import torch.nn.functional as F

import numpy as np

In [ ]:
model_config={
    "model_weights":["../models//fold0_epoch_15_val_rocauc_0.992892_loss_0.053698.pth",
                    "../models//fold1_epoch_17_val_rocauc_0.988631_loss_0.059439.pth",
                    "../models//fold2_epoch_19_val_rocauc_0.991760_loss_0.052413.pth ",
                    "../models//fold3_epoch_19_val_rocauc_0.994017_loss_0.055336.pth"],
    
    
    "model_structure": "tf_efficientnet_b0_ns",
    "model_head_dim": 1280,

    "sub_file":"../../sample_submission.csv",
    "test_dir":"../../test",
    "batch_size":128*2,
    "num_thread":4
}

In [ ]:

import sys
sys.path.append('.')
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import Parameter





import timm



class Net(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()





        self.model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)



        self._avg_pooling = nn.AdaptiveAvgPool2d(1)



        self._fc = nn.Linear(1280 , num_classes, bias=True)

    def forward(self, X):

        #do preprocess


        bs = X.size(0)
        X=torch.cat([X,X,X],dim=1)
        # Convolution layers
        x = self.model.forward_features(X)
        fm = self._avg_pooling(x)
        fm = fm.view(bs, -1)


        x = self._fc(fm)

        return x





model=Net()


In [ ]:
def get_test_df():
    test = pd.read_csv(model_config['sub_file'])

    def get_test_file_path(image_id):
            return model_config['test_dir'] + "/{}/{}.npy".format(
                image_id[0], image_id
            )
    test['file_path'] = test['id'].apply(get_test_file_path)
    return test

test=get_test_df()



class TestDataIter():
    def __init__(self, df):

        self.df = df
        self.resize_trans=A.Resize(height=512, width=512)
        

        
    def __getitem__(self, item):

        return self.single_map_func(self.df.iloc[item])

    def __len__(self):


        return len(self.df)

   
    def single_map_func(self, dp):
        """Data augmentation function."""
        ####customed here
        fname = dp['file_path']
        

        img = np.load(fname).astype(np.float32)[[0,2,4]]  # shape: (3, 273, 256)

        ## to hwc
        img = np.transpose(img,axes=[1,2,0])

        

        ## to chw
        img = np.transpose(img, axes=[2,0,1])

        img = np.vstack(img)  # shape: (819, 256)

        transformed = self.resize_trans(image=img)
        img = transformed['image']

        img = np.expand_dims(img, 0)
        
        return img


def get_dataiter():


    test_generator = TestDataIter(test)
    print(len(test_generator))
    test_ds = DataLoader(test_generator,
                             model_config['batch_size'],
                             num_workers=model_config['num_thread'],
                             shuffle=False)

    return test_ds

test_ds=get_dataiter()

len(test_ds)

In [ ]:
def main():
    
    
    device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    text_preds = []

    pres = []
    weights=model_config["model_weights"]
    
    for fold in range(len(weights)):
        print(' infer with weight %s' % (weights[fold]))
        model.load_state_dict(torch.load(weights[fold], map_location=device))

        model.eval().to(device)
        
        one_fold_predictin=[]
        
        with torch.no_grad():
            for images in tqdm(test_ds):
                data = images.to(device).float()
                
                predictions = model(data)
                predictions = torch.sigmoid(predictions).detach().cpu().numpy()
                
                one_fold_predictin.append(predictions)

        one_fold_predictin=np.concatenate(one_fold_predictin).reshape(-1)
        pres.append(one_fold_predictin)
    
    test['target']=np.mean(pres,0)
    
    test[['id','target']].to_csv('submission.csv',index=False)
    

if __name__=='__main__':
    main()